# Import Library

In [29]:
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from urllib.parse import quote_plus

import time
from datetime import datetime
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

from urllib.error import HTTPError

# Create Empty List & Parameter Setting

In [2]:
url = 'http://www.instagram.com'
path = 'C:\\Users\\rectworks\\Downloads\\chromedriver_win32\\chromedriver.exe'

SCROLL_PAUSE_TIME = 3.0
ID = 'hanshow113'
PW = 'wlsldjtm12!'

post_link = []
popularPost_len = []

id_list = []
like_list = []
tag_list = []
link_list = []
date_list = []
month_list = []
day_list = []


# Define Function

In [3]:
def instagram_login(id, pw):
    driver.get(url)
    driver.implicitly_wait(5)
    driver.find_element_by_name('username').send_keys(id)    # id 입력
    elem_pw = driver.find_element_by_name('password')    # pw 입력
    elem_pw.send_keys(pw)
    elem_pw.submit()
    
    driver.implicitly_wait(5)    # 파싱될 때까지 5초 기다림 (미리 완료되면 waiting 종료됨)
    driver.find_element_by_class_name('cmbtv').click()    # 비밀번호 저장하지 않음
    
    driver.implicitly_wait(5)
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()    # 알림설정 무시

In [4]:
def main_search(keyword):
    search = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')
    search.clear()
    search.send_keys(keyword)
    search_list1 = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div[2]/div/a[1]')
    search_list1.click()

In [5]:
def scroll_page():
    post_link.clear()
    popularPost_len.clear()
    while True:
        pageString = driver.page_source   # page_source : 현재 렌더링된 페이지의 Elements를 모두 가져옴
        bsObj = bs(pageString, 'lxml')

        for postline in bsObj.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
            a_len = len(postline.select('a'))
            popularPost_len.append(a_len)
            # 인스타그램 게시물은 행별로 최대 3개까지 확인할 수 있는데, 최근게시물이나 마지막 게시물은 1,2개가 나올 수도 있어서 len 지정
            for post in range(a_len):   
                item = postline.select('a')[post]
                link = item.attrs['href']
                if link not in post_link:   # 스크롤을 내리고 중복된 것을 제거하지 않고 누적시키기 때문에 없는 것만 추가
                    post_link.append(link)

        last_height = driver.execute_script('return document.body.scrollHeight')   # 자바스크림트로 스크롤 길이를 넘겨줌
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # selenium에서 scroll 기능 사용
        time.sleep(SCROLL_PAUSE_TIME)
        # 프로세스 자체를 지정시간동안 기다려줌(무조건 지연)
        #driver.implicitly_wait(SCROLL_PAUSE_TIME)
        # 브라우저 엔진에서 파싱되는 시간을 기다려줌(요소가 존재하면 지연없이 코드 실행)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            #driver.implicitly_wait(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:
                break
            else:
                last_height = new_height
                continue

In [6]:
def popularPostSum():
    popularPost = 0
    for i in range(3):
        popularPost += popularPost_len[i]
    return popularPost

# 인기게시물은 시간순으로 나타나는 것이 아니므로
# 여기에 나타나는 항목들은 시간순으로 데이터프레임 만다는 데서 제외시키기 위한 함수

In [7]:
def to_timestamp(std_date):
    date_string = std_date
    date =datetime.strptime(date_string, "%Y.%m.%d")
    time_tuple = date.timetuple()
    std_timestamp = time.mktime(time_tuple)
    return int(std_timestamp)
    

# 날짜 옵션 추가

In [8]:
def date_based_crawling(std_date):
    id_list.clear()
    like_list.clear()
    tag_list.clear()
    link_list.clear()
    date_list.clear()
    month_list.clear()
    day_list.clear()
    
    
    post_linkx = post_link[popularPostSum():]    # 스크롤 내리면서 모은 데이터 중 인기 게시물은 제외시킴
    num_of_postx = len(post_linkx)
    

    std_timestamp = to_timestamp(std_date)
    # 기준 날짜를 지나면 크롤링이 멈추도록 작동
    
    for i in tqdm_notebook(range(num_of_postx)):
        req = Request("https://www.instagram.com" + post_linkx[i], headers={'User-Agent': 'Mozila/5.0'})
        postpage = urlopen(req).read()
        

        post_body = bs(postpage, 'lxml', from_encoding='utf-8')
        post_core = post_body.find('meta', attrs={'property':"og:description"})
        contents = post_core['content']

        # break할 시간 비교
        posttxt = str(postpage)    
        timestamp = int(posttxt[posttxt.find('taken_at_timestamp')+20 : posttxt.find('taken_at_timestamp')+30])
        
        
        if (std_timestamp > timestamp) and (to_timestamp(date_list[-1][:10]) - timestamp) > 86400 : 
            pass
        if (std_timestamp > timestamp) and (to_timestamp(date_list[-1][:10]) - timestamp) < 86400 : 
            break
        # 게시글 중간에 오래된 게시글이 올라오는 경우가 있어서 이전 게시글과 하루 이상 차이(timestamp 86400)가 나면
        # outlier로 간주하고 pass
        
        
        # 시간
        date_list.append(datetime.fromtimestamp(timestamp).strftime('%Y.%m.%d %H:%M'))
        month_list.append(datetime.fromtimestamp(timestamp).strftime("%m"))
        day_list.append(datetime.fromtimestamp(timestamp).strftime("%d"))

        # 개별 링크 리스트 
        link_list.append("https://www.instagram.com" + post_linkx[i])   
        
        # 좋아요
        try:
            likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
        except:
            likes=0  # 좋아요 가 아니라 조회수로 표시되는 경우도 있어 이런 경우는 0으로 표시
        like_list.append(likes)

        # 개별 계정
        if "@" and ")" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find(")")]
        elif "shared a post on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('shared a post on Instagram')]                
        elif "shared a photo on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('shared a photo on Instagram')]                
        elif "@" and ")" not in contents and "on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
        else :
            personal_id = contents[1: contents.find(' posted on')]
        id_list.append(personal_id)

        '''    
        (@personal_id) on instagram, @persoanlid posted on instagram, personal_id on instgram 등의 형태로 meta 데이터에 표시되기
        때문에 여러 형식별 id 추출 if문 수행
        '''

        # 해시태그
        tag_list.append([])
        for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
            hashtags = tag_content['content'].rstrip(',')
            tag_list[i].append(hashtags)

# 날짜 옵션 제거

In [31]:
def Crawling():
    id_list.clear()
    like_list.clear()
    tag_list.clear()
    link_list.clear()
    date_list.clear()
    month_list.clear()
    day_list.clear()

    post_linkx = post_link[popularPostSum():]    # 스크롤 내리면서 모은 데이터 중 인기 게시물은 제외시킴
    num_of_postx = len(post_linkx)

    for i in tqdm_notebook(range(num_of_postx)):

        # 개별 링크 리스트    
        req = Request("https://www.instagram.com" + post_linkx[i], headers={'User-Agent': 'Mozila/5.0'})
        try:
            postpage = urlopen(req).read()
        except HTTPError as e:
            print('HTTPError Occured')
            print(e.code)
            print(e.reason)
            pass
        
        post_body = bs(postpage, 'lxml', from_encoding='utf-8')
        post_core = post_body.find('meta', attrs={'property':"og:description"})
        contents = post_core['content']

        # 시간    
        posttxt = str(postpage)    
        timestamp = int(posttxt[posttxt.find('taken_at_timestamp')+20 : posttxt.find('taken_at_timestamp')+30])
        date_list.append(datetime.fromtimestamp(timestamp).strftime('%Y/%m/%d %H:%M'))
        month_list.append(datetime.fromtimestamp(timestamp).strftime("%m"))
        day_list.append(datetime.fromtimestamp(timestamp).strftime("%d"))
        
        # 개별 링크
        link_list.append("https://www.instagram.com" + post_linkx[i])
        
        # 좋아요
        try:
            likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
        except:
            likes=0  # 좋아요 가 아니라 조회수로 표시되는 경우도 있어 이런 경우는 0으로 표시
        like_list.append(likes)

        # 개별 계정
        if "@" and ")" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find(")")]
        elif "shared a post on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('shared a post on Instagram')]                
        elif "shared a photo on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('shared a photo on Instagram')]                
        elif "@" and ")" not in contents and "on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
        else :
            personal_id = contents[1: contents.find(' posted on')]
        id_list.append(personal_id)

        '''    
        (@personal_id) on instagram, @persoanlid posted on instagram, personal_id on instgram 등의 형태로 meta 데이터에 표시되기
        때문에 여러 형식별 id 추출 if문 수행
        '''

        # 해시태그
        tag_list.append([])
        for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
            hashtags = tag_content['content'].rstrip(',')
            tag_list[i].append(hashtags)

In [42]:
def To_df():
    insta_dict = {'날짜':date_list,'월':month_list,'일':day_list,'계정':id_list,
          '좋아요':like_list,'해시태그':tag_list,'링크':link_list}
    df = pd.DataFrame(insta_dict)
    return df

In [44]:
   
def sorted_df():
    column = input("정렬 기준 : ")
    sorted_df = df.sort_values(by=[column], axis=0, ascending=False)
    return sorted_df

# Operation check

In [9]:
driver = webdriver.Chrome(path)

instagram_login(ID, PW)

In [10]:
main_search('#춘천')

In [11]:
scroll_page()

In [12]:
len(post_link)

10710

In [24]:
post_link[695]

'/p/CDI0csZJFgN/'

In [77]:
req = Request("https://www.instagram.com" + post_link[11], headers={'User-Agent': 'Mozila/5.0'})
postpage = urlopen(req).read()


post_body = bs(postpage, 'lxml', from_encoding='utf-8')
post_core = post_body.find('meta', attrs={'property':"og:description"})
contents = post_core['content']
contents

'이선화 shared a post on Instagram: “#황토집숯불닭갈비…” • Follow their account to see 1,396 posts.'

In [78]:
post_core

<meta content="이선화 shared a post on Instagram: “#황토집숯불닭갈비…” • Follow their account to see 1,396 posts." property="og:description"/>

In [15]:
date_based_crawling('2020.07.01')

In [32]:
Crawling()

HTTPError Occured
404
Not Found
HTTPError Occured
404
Not Found
HTTPError Occured
404
Not Found
HTTPError Occured
404
Not Found
HTTPError Occured
404
Not Found
HTTPError Occured
404
Not Found
HTTPError Occured
404
Not Found
HTTPError Occured
404
Not Found



In [41]:
To_df()

,날짜,월,일,계정,좋아요,해시태그,링크
0,2020/07/28 10:03,07,28,89_baek.c.y: “#춘천 #춘천여행 #야경 #꾸르잼👍 #존맛탱👍#오늘부터 #...,0,"[펌프연습, 존맛탱👍, 춘천여행, 춘천, 오늘부터, 야경, 사격연습, 꾸르잼👍]",https://www.instagram.com/p/CDKqeOhDxmj/
1,2020/07/28 10:02,07,28,v___pearl,22,"[초록초록, 춘천, 일상, 해피초원목장, 날씨맑음, 춘천여행]",https://www.instagram.com/p/CDKqXXwFoFQ/
2,2020/07/28 10:01,07,28,chozzang_nim,0,"[춘천, 梅雨入り, 보케, 장마, ボケ, chuncheon]",https://www.instagram.com/p/CDKqPztFseP/
3,2020/07/28 10:00,07,28,doggroomer2619119,26,"[충무로애견, 춘천애견, 선팔하면맞팔, 애견미용사, 춘천충무로애견, 칠전동애견용품,...",https://www.instagram.com/p/CDKqNFLlQNK/
4,2020/07/28 10:00,07,28,춘천시 대신 전해드립니다. shared a photo on Instagram: “호...,0,"[춘천가볼만한곳, 로맨틱스튜디오, 의암호, 춘천여행, 물레길, 춘천카누, 춘천, 춘...",https://www.instagram.com/p/CDKqLcAhTbo/
...,...,...,...,...,...,...,...
10696,2020/07/18 17:11,07,18,maison_de_seul,12,"[사진, 일상, 군인가족, 춘천댁, 메리골드, 춘천, 해바라기, 더워, 다둥이네집,...",https://www.instagram.com/p/CCxrkaflYPw/
10697,2020/07/18 17:11,07,18,organic_cafe_,177,"[백일홍축제, 춘천꽃밭]",https://www.instagram.com/p/CCxrh-XJsH9/
10698,2020/07/18 17:09,07,18,maison_de_seul,15,"[군인가족, 핵더위, 엄청차가움, 유기농카페, 다둥이네집, 핵신남, 육아맘, 강원도...",https://www.instagram.com/p/CCxrQWrlPuV/
10699,2020/07/18 17:08,07,18,s2onghun1981,7,"[춘천, 카페나스, 날씨가, 소양강댐, 강산숯불닭갈비, 급드라이브]",https://www.instagram.com/p/CCxrLioFfG3/


In [40]:
sorted_df()

정렬 기준 : 날짜


,날짜,월,일,계정,좋아요,해시태그,링크
0,2020/07/28 10:03,07,28,89_baek.c.y: “#춘천 #춘천여행 #야경 #꾸르잼👍 #존맛탱👍#오늘부터 #...,0,"[펌프연습, 존맛탱👍, 춘천여행, 춘천, 오늘부터, 야경, 사격연습, 꾸르잼👍]",https://www.instagram.com/p/CDKqeOhDxmj/
1,2020/07/28 10:02,07,28,v___pearl,22,"[초록초록, 춘천, 일상, 해피초원목장, 날씨맑음, 춘천여행]",https://www.instagram.com/p/CDKqXXwFoFQ/
2,2020/07/28 10:01,07,28,chozzang_nim,0,"[춘천, 梅雨入り, 보케, 장마, ボケ, chuncheon]",https://www.instagram.com/p/CDKqPztFseP/
4,2020/07/28 10:00,07,28,춘천시 대신 전해드립니다. shared a photo on Instagram: “호...,0,"[춘천가볼만한곳, 로맨틱스튜디오, 의암호, 춘천여행, 물레길, 춘천카누, 춘천, 춘...",https://www.instagram.com/p/CDKqLcAhTbo/
3,2020/07/28 10:00,07,28,doggroomer2619119,26,"[충무로애견, 춘천애견, 선팔하면맞팔, 애견미용사, 춘천충무로애견, 칠전동애견용품,...",https://www.instagram.com/p/CDKqNFLlQNK/
...,...,...,...,...,...,...,...
9436,2019/07/15 13:24,07,15,insta_yoonji,14,[춘천],https://www.instagram.com/p/Bz7ISUCJuRa/
1784,2019/06/24 10:35,06,24,yul.love.2016,20,"[진정한, 힐링, 제이드가든수목원, 제이드가든, 춘천]",https://www.instagram.com/p/BzEwMtvBXhU/
9973,2018/09/26 21:35,09,26,only_ho7,189,"[춘천, 강원fc, selfies, 패트리어트, 버스, 먹스타그램, 수원삼성, 원정...",https://www.instagram.com/p/BoMISLmBI-2/
4067,2018/08/25 13:33,08,25,the_cafelover,110,"[소통, 맛스타그램, foodie, 갬성카페, 소양강스카이워크, stable, 일상...",https://www.instagram.com/p/Bm43rYpnmv9/
